In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import datetime
import itertools
from datetimerange import DateTimeRange
import pickle

In [2]:
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI')
os.getcwd()

'/home/candela/Documents/greeneHill/membershipReportsCIVI/github/greeneHill'

In [3]:
os.listdir()

['civiMemberReportsExplore.ipynb',
 'prodDropboxII.py',
 '.git',
 'civiActivityReport.ipynb',
 'dropboxController.py',
 'requirements.txt',
 'inspect.csv',
 'Untitled.ipynb',
 'select_activity_report.csv']

In [2]:
#activityReport = pd.read_csv('./membershipReportingLogic SampleReports/select_activity_report.csv')
activityReport = pd.read_csv('./select_activity_report.csv')

In [3]:
activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]

In [4]:
activityReport = activityReport.assign(Activity_Date_DT_act = pd.to_datetime(activityReport['Activity_Date_act'], format = '%Y-%m-%d %H:%M'))

In [5]:
activityReport = activityReport.assign(Activity_Date_DT_act = activityReport['Activity_Date_DT_act'].dt.date)

In [8]:
activityReport.shape

(7645, 10)

In [6]:
#email selection logic: if Source Email is an admin, then select Target Email
#alternatives: where Target Email is blank, select Source Email
activityReport = activityReport.assign(email_grouping = activityReport.apply(lambda x: x['Target_Email_act'] if pd.notnull(x['Target_Email_act']) else x['Source_Email_act'], axis = 1))

In [ ]:
#investigate trial-related activity to ensure that there are appropriate labels for trial associations later
status_lookup = {'Active to General Leave':'general leave', 'to Cancelled':'cancelled', 'General Leave to New':'winback',  
                    'New to Expired':'trial expiration', 'to Deactivated':'deactivated', 'Deactivated to General Leave':'general leave','Deactivated to Expired':'trial expiration', 'to Expired':'trial expiration','Cancelled to Active':'winback','Active to Reactivated':'technical reactivation','to Parental Leave':'parental leave', 'General Leave to Active':'winback', 'Cancelled to Re-activated':'winback', 'to General Leave': 'general leave',
                    'Deactivated to Re-activated':'winback', 'Deactivated to Active':'winback','to Medical Leave':'medical leave',
                      'General Leave to Re-activated':'winback','Medical Leave to Re-activated':'winback', 'Expired to Active':'trial conversion', 'Parental Leave to Active':'winback','Expired to New':'trial conversion','Active to Re-activated':'technical reactivation','Disability Leave to Active':'winback','Active to Care Giving Leave':'care giving leave','Active to New':'technical activation','Expired to Re-activated':'trial conversion','New to Re-activated':'technical re-activation','Medical Leave to Active':'winback','Care Giving Leave to Active':'winback','Re-activated to New':'technical activation'}

#pd.DataFrame.from_dict(status_lookup, orient = 'index', columns = ['raw_status','treated_label'])
sl_df = pd.DataFrame.from_dict(status_lookup, orient = 'index').reset_index()
sl_df.columns = ['raw_status','treated_label']
sl_df.sort_values('raw_status')

In [7]:
class Member_transactions(object):
    # a utility to process Member dictionaries: both status and type
    #I need both the prevailing membership type as of a particular date and the status/trigger that led to it; 
    #it might be nice to place a range of beg and end dates for a membership type
    #set the class attributes
    #perhaps can employ a class inheritence to use this blueprint for status lookups
    Subject_act = {'Trial Membership - ':'trial','Trial Membership - Status: New':'trial','Zucchini Plan -':'6 mo trial','Zucchini Plan - Status: New':'6 mo trial','to Zucchini Plan':'6 mo trial','Lettuce Plan -':'lettuce', 'Avocado Plan -':'avocado','to Avocado Plan':'avocado','Park Slope Member -':'park slope','Park Slope Member - Status: New':'park slope','Apple Plan -':'apple','to Apple Plan':'apple', 'Bushwick Member -':'bushwick','Bushwick Member - Status: New':'bushwick','Carrot Plan - ':'carrot', 'Household Plan - ':'household', 'to Lettuce Plan':'lettuce','to Carrot Plan':'carrot','to Bushwick Member':'bushwick'}
    
    status_lookup = {'Active to General Leave':'general leave', 'to Cancelled':'cancelled', 'General Leave to New':'winback',  
                    'New to Expired':'trial expiration', 'to Deactivated':'deactivated', 'Deactivated to General Leave':'general leave','Deactivated to Expired':'trial expiration', 'to Expired':'trial expiration','Cancelled to Active':'winback','Active to Reactivated':'technical reactivation','to Parental Leave':'parental leave', 'General Leave to Active':'winback', 'Cancelled to Re-activated':'winback', 'to General Leave': 'general leave',
                    'Deactivated to Re-activated':'winback', 'Deactivated to Active':'winback','to Medical Leave':'medical leave',
                      'General Leave to Re-activated':'winback','Medical Leave to Re-activated':'winback', 'Expired to Active':'trial conversion', 'Parental Leave to Active':'winback','Expired to New':'trial conversion','Active to Re-activated':'technical reactivation','Disability Leave to Active':'winback','Active to Care Giving Leave':'care giving leave','Active to New':'technical activation','Expired to Re-activated':'trial conversion','New to Re-activated':'technical re-activation','Medical Leave to Active':'winback','Care Giving Leave to Active':'winback','Re-activated to New':'technical activation'}
    #second/higher level lookup: to be conducted after lower level lookups
    member_abstract={'trial':'trial','lettuce':'full member','6 mo trial':'trial','avocado':'full member',
                     'carrot':'full member','bushwick':'full member','park slope':'full member','apple':'full member'}
    lookup_dict_finder = {'type':Subject_act, 'status':status_lookup,'type_abstract':member_abstract}                     
    ''' candidate 'winback' status changes: 
    Cancelled to Re-activated
    Deactivated to Re-activated
    General Leave to Re-activated
    Medical Leave to Re-activated
    '''
    def __init__(self, dicty):
        #accept the transactional dictionary (by member)
        self.dicty = dicty
        self.class_dict = None #placeholder (this will be revised, if need-be, in dict_loop elif statements)
        return None
    
    #returns the key, which I use to look up the value in the lookup dict
    #designed to be multi-faceted (accepts any lookup dict)
    def dictCrawl(self,substring,lookup_dict):
        result_list = [i for i in lookup_dict.keys() if i in substring] 
        if len(result_list) == 0:
            return None
        else:
            #return the value from the lookup dict, after honing in on the matching key to the substring
            return lookup_dict[result_list[0]]
    
    #loop through ea key of dicty, pass through a string search function and return the corresponding dict value 
#from the initial vetting dict into the new results dict
#believe that the below also appends a seq number to the value retrieved from the dict
# have to swap the key of 
    def dict_loop(self,lookup_dict_key):
        
        def apply_lookup(base_dict, lookup_key):
            #print(f'from within Member_transactions.dict_loop, the relevant dictionary is of length {len(self.dicty)}')
            new_results = {} # will store the keys from the lookuptable and the values from the member dict
            seq_num = 0
            for k,v in base_dict.items(): # dicty = member dictionary
                new_key = self.dictCrawl(k,self.lookup_dict_finder[lookup_key])
                if new_key is None:
                #do something
                    print(f"new member type doesn't match any lookup keys: {k}")
                else:
                #write results to a new dict
                    #print(f"dict_loop of Member_trans function new key is {new_key}")
                    #print(f"dict_loop of Member_trans function new key is {new_key+'_'+str(seq_num)}")
                    new_results[new_key+'_'+str(seq_num)] = v
                    seq_num= seq_num+1
            return new_results
        
        # if/then statements serves as a map of lookup_dict_key to "base" dict
        if lookup_dict_key == 'type_abstract':
            try: 
                #feed the class_dict (set at the elif statements below) into the apply lookup function to convert to abstract dict; 
                self.type_abstract_dict = apply_lookup(self.class_dict,'type_abstract')
            except NameError: #check if class_dict has already been assigned
                raise('the type dict does not yet exist; run dict_loop with lookup key = class')
        elif lookup_dict_key == 'type':
            self.class_dict = apply_lookup(self.dicty,'type')    
            return self.class_dict
        elif lookup_dict_key == 'status': # status should be handled at the class level so that all attributes are attached to same object
            self.class_dict = apply_lookup(self.dicty,'status')
            return self.class_dict
        else: # OK to proceed
            return None
    

In [8]:
class Member(object):
    #cobble together attributes of a particular member by extracting activity from the df
    
    #class attribute, bc never changes; the activity_df object would need to exist before this assignment
    #membership_df = activity_df

    #ideally I store the following attributes of a member: Ea status row carries with it: 1) applicable "type", 2) prevailing "type", 3) "status" lookup value (from Membership_transactions)
    
    def __init__(self, activity_df, email):
        self.email = email
        #instance attribute, but perhaps can be moved to a class attribute
        self.membership_df = activity_df
        
        #can choose to call email_lookup() here
        self.email_lookup()

        self.convertType_toDict() #creates self.dicty_type
        self.convertStatus_toDict() #creates self.dicty_status
        

    def email_lookup(self):
        #search the activity df for all instances of the email: both membership type changes and status changes
        try:
            # membership type change
            self.df = self.membership_df.loc[self.membership_df['email_grouping'].str.strip() ==self.email,:].sort_values(by = 'Activity_Date_DT_act')
            #remove some trivial activities
            self.df2 = self.df.loc[~self.df['Subject_act'].isin(['Status changed from New to Active','Status changed from Pending to New','Status changed from Re-activated to Active']),]
            #write a sequential key to a series on the df: this is used to account for activity (Status or Type) that repeats (ex. multiple trials, etc), because when written to a dict below (convertType_toDict and convertStatus_toDict), the keys must be unique
            self.df2['seq_num'] = range(0,len(self.df2.index))
            #convert to str 
            self.df2 = self.df2.assign(seq_num = self.df2['seq_num'].astype(str))
            #concat two Series
            self.df2 = self.df2.assign(Subject_act = self.df2['Subject_act'].str.cat(self.df2['seq_num'], sep = '_'))

            if self.df2.size == 0:
                #print('no match for email found')
                pass
            else:
                #print('email found')
                pass
        except Exception as e:
            # return something if the member email is not found in the activty dataframe
            print(e)
            
    #consider creating a separate class for all the data manipulation and lookup conversions
    def convertType_toDict(self):
        #filter on 'Activity_Type_act': {'Change Membership Status','Membership Signup','Change Membership Type'}
        #convert two series from the member df to a dict for future mapping/conversion
        dicty = {}
        #select for Membership types
        self.type_dict = self.df2.loc[self.df2['Activity_Type_act'].isin(['Membership Signup','Change Membership Type']),['Subject_act','Activity_Date_DT_act']]
        for i in self.type_dict.values:
            dicty[i[0]] = i[1]
        self.dicty_type_member = dicty
        return self.dicty_type_member
    
    def convertStatus_toDict(self):
        #filter on 'Activity_Type_act': {'Change Membership Status','Membership Signup','Change Membership Type'}
        #convert two series from the member df to a dict for future mapping/conversion
        dicty = {}
        #select for Membership types
        for i in self.df2.loc[self.df2['Activity_Type_act'].isin(['Change Membership Status']),['Subject_act','Activity_Date_DT_act']].values:
            dicty[i[0]] = i[1]
        self.dicty_status = dicty
        return self.dicty_status
    
    def interact_member_transactions(self):
        #interact with the Member_transactions class returning normalized dictionaries for "type" and "status"
        try:
            member_trans = Member_transactions(self.dicty_type_member)
            #dict_loop will likely return a dict
            dict_type = member_trans.dict_loop('type')
            #print(f"length of dicty_type_member is {len(self.dicty_type_member)}, length of dict_status is {len(dict_type)}")
            assert len(self.dicty_type_member) == len(dict_type), "lengths of self.type_dict and dict_status in class Member don't match"
        except Exception as e:
            print("error generated in class Member.interact_member_transactions on 'type' process")
            print(e)

        try:
            member_trans = Member_transactions(self.dicty_status)
            dict_status = member_trans.dict_loop('status')
        except Exception as e:
            print("error generated in class Member.interact_member_transactions on 'status' process")
            print(e)

        return dict_type,dict_status

    def normalize_member_class(self):
        #will lookup whether a member is full, trial, et al
        return None

### Handling trial expirations and conversions
Contemplating applying a ruleset: if the trial-related activity (cancellation, conversion, etc) occurs within the trial window (can be 1,2,6 months), then the 'type' activity is assigned to that trial
If there is no subsequent 'type' initiation/activation after the latest trial, then assign the 'status' to the last trial -> this will cover activity outside of trial date range

- Consider creating a trial rollover whereby a member rolls over from a 1 to 6 month trial

CASES: 
- 405sarah@gmail.com enrolls in a two month trial, which I need to relabel at the Member level
- 009janaki@gmail.com rolls a 1 month trial over to a 6 month trial INSIDE the window of the first 1 mo period
- aaikolord@gmail.com converts a trial to a full membership after trial expiration - implement a time cap for conversion
- aatiell@gmail.com: accurately encoded the status type to trial conversion
- abrennan@wesleyan.edu: re-code for 2 month trial expiration
- adam.b.neese@gmail.com: case study of a legit 1 mo trial
- adwait.spotify@gmail.com: may have simultaneously enrolled in two trials (one and two month)
- aef@fastmail.com: rolled from 1 month to 6 month
- aelewis22@yahoo.com: two trials and two conversions to full
- aidan.l.ling@gmail.com: conversion after trial expiration
- akari@tribecapediatrics.com: logged two identical trials on the same date (effectively one should be ignored)
- alembeck5@gmail.com: conversion to full within trial window and no expiration recorded for the corresponding trial
- alina@alinarancier.com: convert after trial expiration, and later go on to have many winbacks

### Regarding trials
Easiest route may be to associate all activity to trials having a start date BEFORE the date of activity; this may handle cases of multiple trials and actions in an effort to avoiding associating activity to the wrong trial

In [10]:
class Trial_pattern(object):
    #purpose of this class is to create the infrastructure to properly evaluate trial members: relate status records to the appropriate membership types, including both expirations and conversions
    #non-trials are more straightforward because the status change will correspond to the prevailing "type" on activity date
    #this version of Trial_pattern class differs from the above in that it is suited to dataframe input as opposed to the dictionaries from the intermittent output of the Member class
    #designed to ingest the entire Member Type df at once, and set a "theoretical" expiration for ea trial based on Travis' critera:
    '''
    Travis: I can't find the exact date trials were changed from 30 days to 60, but it was around December 20th 2021.
    
    Me: So to be clear, are you saying that all trials (not Zucchini) prior to December '21 should have an expiration of one month and those initiated after that date should have a length of 2 months?
     
    '''
    threshold = datetime.datetime.strptime('2021-12-20', '%Y-%m-%d')

    def __init__(self, df):
        #pull both the 'type' and 'status' dictionaries from a member
        self.df_orig = df.copy()
        #this will change the original df object as well
        self.df_orig['trial_expiration'] = np.nan
        #add 'trial expiration' field to dataframe
        self.df_trial = df.loc[df['type'].str.contains('trial'),:]

        #self.process()
        #class variable switch date for trials
        

    def loop_thru(self):
        # for ea record, select the expiration date by date of trial enrollment; per Travis 12/20/21 is the switch-over date from 1 mo to 2 mo trial lengths

        def define_expirations(df_row):
        # associate ea expiration entry in status changes to the appropriate trial (can be multiple trials for a member)
        # the value of this dict can be added directly to a datetime object, ex. datetime.datetime(2020,3,7) + value
        # a typical matching record will have {'trial type': datetime.date(2019, 12, 5)}
            time_deltas = {'1 mo trial':datetime.timedelta(days = 30), '2 mo trial':datetime.timedelta(days = 60), '6 mo trial': datetime.timedelta(days = 180)}
        
            def assign_expiry(df_row):
                if '6 mo trial' in df_row['type'] :
                    trial_expiration = '6 mo trial'
                elif df_row['start_dt'] < Trial_pattern.threshold:
                    trial_expiration = '1 mo trial'
                elif df_row['start_dt'] >= Trial_pattern.threshold:
                    trial_expiration = '2 mo trial'
                else:
                    None
                return trial_expiration
            #define_expirations returns below value
            return df_row['start_dt']+time_deltas[assign_expiry(df_row)]
        #change the class variable
        self.df_trial['trial_expiration'] = self.df_trial.apply(define_expirations, axis = 1)

        return None
        
    def process(self):
        #call function to alter self.df_trial
        self.loop_thru()

        self.df_non_trial = self.df_orig.loc[~self.df_orig['type'].str.contains('trial'),:]
        #concatenate both dataframes
        return pd.concat([self.df_trial,self.df_non_trial])
            

In [13]:
#print intermittent output of the Member classes in order to determine best step at which to apply the trial functions
aatiell_class = Member(activityReport,'aatiell@gmail.com')
mt,ms = aatiell_class.interact_member_transactions()
print(mt)
print(ms)

{'6 mo trial_0': datetime.date(2021, 12, 12), 'lettuce_1': datetime.date(2022, 6, 27)}
{'trial expiration_0': datetime.date(2022, 6, 11), 'trial conversion_1': datetime.date(2022, 6, 27)}


/home/candela/anaconda3/envs/base_38/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [63]:
# 2nd version of the membership application: create a dataframe of email, type, date range; NOTE: type field is present in both sets of DF (type & status)
# any desired changes to the final df (those inserted into the mysql db) can be done here in addition to the Member and Member_transactions classes
def create_df(mt_v):
    if len(mt_v) == 0:
        return None
    else:
    #TODO will need to account for single row membership activity
        dfj = pd.DataFrame.from_dict(mt_v, orient = 'index').reset_index()
        dfj.columns = ['type','start_dt']
    #convert dtype
        dfj['start_dt'] = pd.to_datetime(dfj['start_dt'])
        dfj.sort_values('start_dt', ascending = True)
        dfj['lead_date'] = dfj['start_dt'].shift(-1)
        dfj['lead_date'] = dfj['lead_date'].apply(lambda x: x-datetime.timedelta(days = 1))
    #replace NaN with today's date
        djf = dfj.assign(lead_date = dfj['lead_date'].fillna(datetime.datetime.today().date().strftime(format = "%Y-%m-%d")))
    # if dataframe is of row length = 1, then the "shifted" column should = today's date
    #create a DateRange object per row
        djf['datetimerange'] = djf.apply(lambda x: DateTimeRange(x['start_dt'],x['lead_date']), axis = 1)
        # sanitize the type field by removing the sequence encoding and storing the abstracted type in its own field
        djf['type_clean'] = djf['type'].str.split(pat = '_', n = 1,expand = True)[0]

        return djf
    
#called independently from create_df()
def compile(unq_email):
    unq_mem_class = Member(activityReport,unq_email)
    mt,ms = unq_mem_class.interact_member_transactions()
    return(mt,ms)

In [64]:
#collect both sets of activity (type and status) into their respective dataframes
# each DF will have the same columns: type (activity name), start_dt, lead_dt (date offset from proceeding activity or today's date), datetimerange, email
list_unq_email = activityReport.email_grouping.unique()
cum_df_type = pd.DataFrame()
cum_df_status = pd.DataFrame()
#iterate through ea email address and process: create a Member object
for i in list_unq_email:
    try:
        #generate both a type and status dict for ea email
        mt,ms = compile(i)
        mt_df = create_df(mt)
        ms_df = create_df(ms)
        if mt_df is None: # no type activity for member
            next
        else:
        #add member email to the df
            mt_df['email'] = i
            cum_df_type = pd.concat([cum_df_type,mt_df], axis = 0)
            
        if ms_df is None: # if no status activity for member, proceed
            next
        else:
        #add member email to the df
            ms_df['email'] = i
            cum_df_status = pd.concat([cum_df_status,ms_df], axis = 0)
        
    except Exception as e:
        print(f'error encountered with {i}; error message {e.args}')   

/home/candela/anaconda3/envs/base_38/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


new member type doesn't match any lookup keys: Status changed from Active to Deceased_0
new member type doesn't match any lookup keys: Status changed from Active to Disability Leave_4
new member type doesn't match any lookup keys: Status changed from Active to Disability Leave_4
new member type doesn't match any lookup keys: Status changed from Active to Disability Leave_6
new member type doesn't match any lookup keys: Status changed from Deactivated to New_2
new member type doesn't match any lookup keys: Status changed from Parental Leave to Re-activated_6
new member type doesn't match any lookup keys: Status changed from General Leave to Care Giving Leave_1
new member type doesn't match any lookup keys: Status changed from Care Giving Leave to Re-activated_2
error generated in class Member.interact_member_transactions on 'type' process
argument of type 'float' is not iterable
error encountered with it@greenehillfood.coop; error message ("local variable 'dict_type' referenced before a

In [14]:
#inspect lengths of each df
print(cum_df_status.shape)
print(cum_df_type.shape)
print(activityReport.shape)
#reconcile the difference btwn sum of status and type df to length of activityReport
print(activityReport.loc[~activityReport['Subject_act'].isin(['Status changed from New to Active','Status changed from Pending to New','Status changed from Re-activated to Active']),].shape)

(3185, 5)
(2702, 5)
(7645, 11)
(5906, 11)


In [68]:
#apply the new trial expiration logic
df = Trial_pattern(cum_df_type)
df2 = df.process()
#df.process().sort_values(['email','start_dt']).to_csv('./inspect.csv', index = False)

/home/candela/anaconda3/envs/base_38/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [69]:
#email is not unique; one record per continuous membership type 
df2 = df2.reset_index(drop = True)
df2.sort_values('email').head(3)

,type,start_dt,lead_date,datetimerange,type_clean,email,trial_expiration
48,6 mo trial_1,2020-01-20,2023-06-25,2020-01-20T00:00:00 - 2023-06-25T00:00:00,6 mo trial,009janaki@gmail.com,2020-07-18
47,trial_0,2019-12-23,2020-01-19,2019-12-23T00:00:00 - 2020-01-19T00:00:00,trial,009janaki@gmail.com,2020-01-22
2106,lettuce_0,2020-07-21,2023-06-25,2020-07-21T00:00:00 - 2023-06-25T00:00:00,lettuce,16defazioe@gmail.com,NaT


In [66]:
cum_df_type = cum_df_type.reset_index(drop = True)
cum_df_type.head(3)

,type,start_dt,lead_date,datetimerange,type_clean,email
0,trial_0,2019-12-05,2021-11-06,2019-12-05T00:00:00 - 2021-11-06T00:00:00,trial,tiffmnix@yahoo.com
1,park slope_1,2021-11-07,2023-06-25,2021-11-07T00:00:00 - 2023-06-25T00:00:00,park slope,tiffmnix@yahoo.com
2,trial_0,2019-12-05,2023-06-25,2019-12-05T00:00:00 - 2023-06-25T00:00:00,trial,mollyfredenberg@gmail.com


At this point I have two tables that I'd like to import into the db: df2 and cum_df_status

In [70]:
#using pickle to persist objects
filename1 = '/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_df2.p'
filename2 = '/home/candela/Documents/greeneHill/membershipReportsCIVI/membership_cum_df_status.p'
filehandler = open(filename1, 'wb') 
pickle.dump(df2, filehandler)
filehandler = open(filename2, 'wb') 
pickle.dump(cum_df_status, filehandler)

In [23]:
import sqlalchemy
#from sqlalchemy import URL

In [71]:
# DEFINE THE DATABASE CREDENTIALS
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)


Connection to the 172.17.0.2 for user root created successfully.


In [73]:
try:
    #frame = cum_df_type.to_sql('mem_status', engine, if_exists='fail')
    frame = df2.to_sql('mem_type', engine, if_exists='fail')
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%'mem_status');   
finally:
    engine.close()

Table mem_status created successfully.


AttributeError: 'Engine' object has no attribute 'close'

### develop the program to join status activity to prevailing member type
- if trial related, the actvity will be assigned to one of: the trial with an active date range that encompasses the activity OR the historical trial that occurred prior to the trial having a start_dt AFTER the activity date of the status 
- once a person becomes a member, they will typically stay in that category, and it's up to me to toggle them active/leave/cancelled depending on status updates
- ultimately I care about: trial conversions, trial expirations/cancellations, 

Attempt to filter for trial members that do not upgrade and observe the permutations of subjects to learn which text is exclusive to trial memberships, so I can properly associate them/dismiss them when building the user journey

In [26]:
cum_df_type.groupby('email').filter(lambda x: any([re.search('trial',i) for i in x['type']])).sort_values(['email','start_dt'])

,type,start_dt,lead_date,datetimerange,email
0,1 mo trial_0,2019-12-23,2020-01-19,2019-12-23T00:00:00 - 2020-01-19T00:00:00,009janaki@gmail.com
1,6 mo trial_1,2020-01-20,2023-05-14,2020-01-20T00:00:00 - 2023-05-14T00:00:00,009janaki@gmail.com
0,1 mo trial_0,2022-02-18,2023-05-14,2022-02-18T00:00:00 - 2023-05-14T00:00:00,39@mailcomfort.com
0,1 mo trial_0,2022-07-05,2022-09-08,2022-07-05T00:00:00 - 2022-09-08T00:00:00,405sarah@gmail.com
1,lettuce_1,2022-09-09,2023-05-14,2022-09-09T00:00:00 - 2023-05-14T00:00:00,405sarah@gmail.com
...,...,...,...,...,...
0,1 mo trial_0,2021-05-05,2021-06-03,2021-05-05T00:00:00 - 2021-06-03T00:00:00,zoehoffmann88@gmail.com
1,lettuce_1,2021-06-04,2023-05-14,2021-06-04T00:00:00 - 2023-05-14T00:00:00,zoehoffmann88@gmail.com
0,1 mo trial_0,2021-09-09,2023-05-14,2021-09-09T00:00:00 - 2023-05-14T00:00:00,zoewohlfeld@gmail.com
0,6 mo trial_0,2021-11-07,2023-05-14,2021-11-07T00:00:00 - 2023-05-14T00:00:00,zoezphillips@gmail.com


In [27]:
#output.groupby('email').filter(lambda x: len(x) == 2)
#trials = output.groupby('email').filter(lambda x: any(x['membership_type'] == 'trial'))
#output.groupby('email').filter(lambda x: any(x['membership_type'] == 'trial'))
#print(trials.shape)
#this version filters the groupby object by TWO conditions
output2 = cum_df_type.groupby('email').filter(lambda x: any([re.search('trial',i) for i in x['type']])).sort_values(['email','start_dt'])
print(output2.shape)
#inspect output of the program
output2.to_csv('./trial_type_sorted.csv', index = False)
cum_df_status.loc[cum_df_status['email'].isin(output2['email'])].sort_values(['email','start_dt']).to_csv('./trial_status_sorted.csv',index = False)

(2207, 5)


In [ ]:
#TODO print Output2 and arrive at the expected shape of the multi conditional (above)
#output2
size_one = output.groupby('email').filter(lambda x: len(x) == 1)
activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),].head()

In [ ]:

#enables reviewing all unique subjects associated to trials (excluding all other member types)
trials_only = activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),['email_grouping','Subject_act','Activity_Date_DT_act']].sort_values(['email_grouping','Activity_Date_DT_act'])
trials_only.to_csv('./trialsOnly.csv', index = False)

#consider only unique values of Subject_act
activityReport.loc[activityReport['email_grouping'].isin(size_one.loc[size_one['membership_type'].str.contains('trial'),'email']),['Subject_act']]['Subject_act'].unique()


In [ ]:
#view output of above
activityReport.loc[activityReport['email_grouping'].isin(output2['email']),].to_csv('./trialMembersFullOutput.csv', index = False)

In [ ]:
# in an attempt to identify status changes that are artifacts of an initial trial, select for all status changes after there is a type change (conversion of trial to other membership type)
# need to remove entries with Status cahnges: "Active to New", "New to Active", "Pending to New"
mult_groups = output.groupby(['email']).filter(lambda x: len(x) >1)
activityReport.loc[(activityReport['email_grouping'].isin(mult_groups['email'])) & (activityReport['email_grouping'].isin(output.loc[output['membership_type'].str.contains('trial'),'email'])),['email_grouping','Activity_Date_DT_act','Subject_act','Activity_Type_act']].sort_values(['email_grouping','Activity_Date_DT_act']).to_csv('./multi_w_trial.csv', index = False)

In [ ]:
pd.Series(activityReport.loc[activityReport['email_grouping'].isin(output2['email']),'Subject_act'].unique())[pd.Series(activityReport.loc[activityReport['email_grouping'].isin(output2['email']),'Subject_act'].unique()).str.contains('Status changed')]

### TODO: start going through event and type use cases, ex. Active to Expired for Trial (when within a two or 6 month period from Trial start)

In [ ]:
#best practice is always to sort this dataframe first
class_df = pd.DataFrame.from_dict(s.dict_loop('status'), orient = 'index', columns = ['start_date']).reset_index()
print(class_df)

In [ ]:
#best practice is always to sort this dataframe first
class_df = pd.DataFrame.from_dict(st.dict_loop('class'), orient = 'index', columns = ['start_date']).reset_index()
print(class_df)

In [ ]:
#lag values
class_df['end_date']=class_df['start_date'].shift(periods=-1, axis = 0)

In [ ]:
class_df.loc[:,'start_date'] = pd.to_datetime(class_df['start_date'],format = '%Y-%m-%d')

In [ ]:
class_df.loc[class_df['end_date'].isna(),'end_date']= datetime.datetime.today().date()

In [ ]:
class_df